In [2]:
# Step 1: Load an experiment configuration from a JSON file
import json
import os
import glob
from scripts.sample_db import SampleDB
from scripts.config_model import Experiment, load_experiment_config, save_experiment_config, update_experiment_config


In [4]:
# Inputs
# Load database csv file
db_file_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'

# root path 
root_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-27\f3'


In [5]:
# Loading experiment configuration
config_file_path = glob.glob(os.path.join(root_path, 'config_*.json'))[0]
print(config_file_path)

config = load_experiment_config(config_file_path)
print("Loaded experiment configuration:")
print(config.paths)


\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-27\f3\config_20220427_RM0008_126hpf_fP3_f3.json
Loaded experiment configuration:
root_path=WindowsPath('.') config_path=WindowsPath('.') trials_path=WindowsPath('.') anatomy_path=WindowsPath('.') em_path=WindowsPath('.')


In [6]:
# Define your directories here
trials_path = os.path.join(root_path, "trials")
anatomy_path = os.path.join(root_path, "anatomy")
raw_trials_path = os.path.join(trials_path,"raw")

# Add to config structure
config.paths.root_path = root_path
config.paths.config_path = config_file_path
config.paths.anatomy_path = anatomy_path
config.paths.trials_path = trials_path

save_experiment_config(config)


\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-27\f3\config_20220427_RM0008_126hpf_fP3_f3.json


In [7]:
# Step 2: Load or create a sample database
sample_db = SampleDB()

try:
    sample_db.load(db_file_path)
    print("Loaded existing sample database.")
except FileNotFoundError:
    print("Sample database not found. Creating a new one.")
    sample_db.save(db_file_path)  # Create a new CSV file with headers

sample = config.sample
print(sample.id)
existing_sample = sample_db.get_sample(sample.id)



Loaded existing sample database.
20220427_RM0008_126hpf_fP3_f3


In [8]:
# Step 3: Check if the sample is already in the database and ask if you want to update it
update=True
if existing_sample:
    print(f"Sample with ID {sample.id} already exists.")
    update = input("Do you want to update the existing sample? (y/n): ").strip().lower() == 'y'
    print(update)
# Ensure initial columns are present
sample_db.add_column('sample_id')
sample_db.add_column('root_path')
sample_db.add_column('config_path')
sample_db.add_column('trials_path')
sample_db.add_column('anatomy_path')
sample_db.add_column('em_path')
sample_db.add_column('update')

# Add the sample data to the database
sample_db.samples[sample.id] = {
    "sample": sample,
    "root_path": root_path,
    "config_path": config_file_path,
    "trials_path": trials_path,
    "anatomy_path": anatomy_path,
    "em_path": "",
    "update": update
}


In [9]:
# Step 4: Save the updated database to a file
sample_db.save(db_file_path)
print(f"Sample database saved to {db_file_path}.")

Sample database saved to \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv.


In [10]:
sample_db

SampleDB(sample_ids=['20220426_RM0008_130hpf_fP1_f3', '20220118_RM0012_124hpf_fP8_f2', '20220427_RM0008_126hpf_fP3_f3'])